## Export .csvs of nodes and edges

In [174]:
# Setup
import json, nltk, pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
import networkx as nx
from plotly.graph_objs import * 
import csv
from nltk import ngrams
import statistics
from decimal import getcontext, Decimal

# Taxonomy of technologies
technologies = [
    {'name': 'javascript', 'category': 'languages', 'subcategory' : 'javascript'},
    {'name': 'html', 'category': 'languages', 'subcategory' : 'html_css'},
    {'name': 'html5', 'category': 'languages', 'subcategory' : 'html_css'},
    {'name': 'css', 'category': 'languages', 'subcategory' : 'html_css'},
    {'name': 'css3', 'category': 'languages', 'subcategory' : 'html_css'},
    {'name': 'html/css', 'category': 'languages', 'subcategory' : 'html_css'},
    {'name': 'sql', 'category': 'languages', 'subcategory' : 'sql'},
    {'name': 'python', 'category': 'languages', 'subcategory' : 'python'},
    {'name': 'java', 'category': 'languages', 'subcategory' : 'java'},
    {'name': 'bash', 'category': 'languages', 'subcategory' : 'bash'},
    {'name': 'shell', 'category': 'languages', 'subcategory' : 'shell'},
    {'name': 'powershell', 'category': 'languages', 'subcategory' : 'powershell'},
    {'name': 'c#', 'category': 'languages', 'subcategory' : 'c#'},
    {'name': 'php', 'category': 'languages', 'subcategory' : 'php'},
    {'name': 'c++', 'category': 'languages', 'subcategory' : 'c++'},
    {'name': 'typescript', 'category': 'languages', 'subcategory' : 'typescript'},
    {'name': 'c', 'category': 'languages', 'subcategory' : 'c'},
    {'name': 'ruby', 'category': 'languages', 'subcategory' : 'ruby'},
    {'name': 'go', 'category': 'languages', 'subcategory' : 'go'},
    {'name': 'assembly', 'category': 'languages', 'subcategory' : 'assembly'},
    {'name': 'swift', 'category': 'languages', 'subcategory' : 'swift'},
    {'name': 'kotlin', 'category': 'languages', 'subcategory' : 'kotlin'},
    {'name': 'r', 'category': 'languages', 'subcategory' : 'r'},
    {'name': 'vba', 'category': 'languages', 'subcategory' : 'vba'},
    {'name': 'objective-c', 'category': 'languages', 'subcategory' : 'objective-c'},
    {'name': 'objective c', 'category': 'languages', 'subcategory' : 'objective-c'},
    {'name': 'scala', 'category': 'languages', 'subcategory' : 'scala'},
    {'name': 'rust', 'category': 'languages', 'subcategory' : 'rust'},
    {'name': 'dart', 'category': 'languages', 'subcategory' : 'dart'},
    {'name': 'elixir', 'category': 'languages', 'subcategory' : 'elixir'},
    {'name': 'clojure', 'category': 'languages', 'subcategory' : 'clojure'},
    {'name': 'webassembly', 'category': 'languages', 'subcategory' : 'webassembly'},
    {'name': 'web assembly', 'category': 'languages', 'subcategory' : 'webassembly'},
    {'name': 'jquery', 'category': 'frameworks', 'subcategory' : 'jquery'},
    {'name': 'react.js', 'category': 'frameworks', 'subcategory' : 'react'},
    {'name': 'react', 'category': 'frameworks', 'subcategory' : 'react'},
    {'name': 'angular', 'category': 'frameworks', 'subcategory' : 'angular'},
    {'name': 'angular.js', 'category': 'frameworks', 'subcategory' : 'angular'},
    {'name': 'asp.net', 'category': 'frameworks', 'subcategory' : 'asp.net'},
    {'name': 'asp', 'category': 'frameworks', 'subcategory' : 'asp'},
    {'name': 'express', 'category': 'frameworks', 'subcategory' : 'express'},
    {'name': 'spring', 'category': 'frameworks', 'subcategory' : 'spring'},
    {'name': 'vue.js', 'category': 'frameworks', 'subcategory' : 'vue.js'},
    {'name': 'vue', 'category': 'frameworks', 'subcategory' : 'vue.js'},
    {'name': 'django', 'category': 'frameworks', 'subcategory' : 'django'},
    {'name': 'flask', 'category': 'frameworks', 'subcategory' : 'flask'},
    {'name': 'laravel', 'category': 'frameworks', 'subcategory' : 'laravel'},
    {'name': 'ruby on rails', 'category': 'frameworks', 'subcategory' : 'ruby on rails'},
    {'name': 'rails', 'category': 'frameworks', 'subcategory' : 'ruby on rails'},
    {'name': 'drupal', 'category': 'frameworks', 'subcategory' : 'drupal'},
    {'name': 'linux', 'category': 'platforms', 'subcategory' : 'linux'},
    {'name': 'windows', 'category': 'platforms', 'subcategory' : 'windows'},
    {'name': 'docker', 'category': 'platforms', 'subcategory' : 'docker'},
    {'name': 'android', 'category': 'platforms', 'subcategory' : 'android'},
    {'name': 'aws', 'category': 'platforms', 'subcategory' : 'aws'},
    {'name': 'macos', 'category': 'platforms', 'subcategory' : 'macos'},
    {'name': 'slack', 'category': 'platforms', 'subcategory' : 'slack'},
    {'name': 'raspberry pi', 'category': 'platforms', 'subcategory' : 'raspberry pi'},
    {'name': 'wordpress', 'category': 'platforms', 'subcategory' : 'wordpress'},
    {'name': 'ios', 'category': 'platforms', 'subcategory' : 'ios'},
    {'name': 'google cloud', 'category': 'platforms', 'subcategory' : 'google cloud platform'},
    {'name': 'google cloud platform', 'category': 'platforms', 'subcategory' : 'google cloud platform'},
    {'name': 'gcs', 'category': 'platforms', 'subcategory' : 'google cloud platform'},
    {'name': 'gcp', 'category': 'platforms', 'subcategory' : 'google cloud platform'},
    {'name': 'azure', 'category': 'platforms', 'subcategory' : 'azure'},
    {'name': 'arduino', 'category': 'platforms', 'subcategory' : 'arduino'},
    {'name': 'heroku', 'category': 'platforms', 'subcategory' : 'heroku'},
    {'name': 'kubernetes', 'category': 'platforms', 'subcategory' : 'kubernetes'},
    {'name': 'ibm cloud', 'category': 'platforms', 'subcategory' : 'ibm cloud'},
    {'name': 'watson', 'category': 'platforms', 'subcategory' : 'watson'},
    {'name': 'mysql', 'category': 'databases', 'subcategory' : 'mysql'},
    {'name': 'postgresql', 'category': 'databases', 'subcategory' : 'postgresql'},
    {'name': 'postgressql', 'category': 'databases', 'subcategory' : 'postgresql'},
    {'name': 'postgres', 'category': 'databases', 'subcategory' : 'postgresql'},
    {'name': 'sql server', 'category': 'databases', 'subcategory' : 'sql server'},
    {'name': 'sqlite', 'category': 'databases', 'subcategory' : 'sqlite'},
    {'name': 'sqllite', 'category': 'databases', 'subcategory' : 'sqlite'},
    {'name': 'sql lite', 'category': 'databases', 'subcategory' : 'sqlite'},
    {'name': 'mongodb', 'category': 'databases', 'subcategory' : 'mongodb'},
    {'name': 'redis', 'category': 'databases', 'subcategory' : 'redis'},
    {'name': 'mariadb', 'category': 'databases', 'subcategory' : 'mariadb'},
    {'name': 'oracle', 'category': 'databases', 'subcategory' : 'oracle'},
    {'name': 'elasticsearch', 'category': 'databases', 'subcategory' : 'elasticsearch'},
    {'name': 'firebase', 'category': 'databases', 'subcategory' : 'firebase'},
    {'name': 'dynamodb', 'category': 'databases', 'subcategory' : 'dynamodb'},
    {'name': 'cassandra', 'category': 'databases', 'subcategory' : 'cassandra'},
    {'name': 'couchbase', 'category': 'databases', 'subcategory' : 'couchbase'},
    {'name': 'node.js', 'category': 'others', 'subcategory' : 'node.js'},
    {'name': 'node', 'category': 'others', 'subcategory' : 'node.js'},
    {'name': '.net', 'category': 'others', 'subcategory' : '.net'},
    {'name': '.net core', 'category': 'others', 'subcategory' : '.net core'},
    {'name': 'pandas', 'category': 'others', 'subcategory' : 'pandas'},
    {'name': 'unity 3d', 'category': 'others', 'subcategory' : 'unity 3d'},
    {'name': 'react native', 'category': 'others', 'subcategory' : 'react native'},
    {'name': 'tensorflow', 'category': 'others', 'subcategory' : 'tensorflow'},
    {'name': 'ansible', 'category': 'others', 'subcategory' : 'ansible'},
    {'name': 'cordova', 'category': 'others', 'subcategory' : 'cordova'},
    {'name': 'xamarin', 'category': 'others', 'subcategory' : 'xamarin'},
    {'name': 'apache spark', 'category': 'others', 'subcategory' : 'apache spark'},
    {'name': 'hadoop', 'category': 'others', 'subcategory' : 'hadoop'},
    {'name': 'unreal engine', 'category': 'others', 'subcategory' : 'unreal engine'},
    {'name': 'flutter', 'category': 'others', 'subcategory' : 'flutter'},
    {'name': 'torch/pytorch', 'category': 'others', 'subcategory' : 'torch/pytorch'},
    {'name': 'torch', 'category': 'others', 'subcategory' : 'torch/pytorch'},
    {'name': 'pytorch', 'category': 'others', 'subcategory' : 'torch/pytorch'},
    {'name': 'puppet', 'category': 'others', 'subcategory' : 'puppet'},
    {'name': 'chef', 'category': 'others', 'subcategory' : 'chef'},
    {'name': 'cryengine', 'category': 'others', 'subcategory' : 'cryengine'}]

# Store multi-word expressions in `technologies` in list `mwe` so significance of, e.g. 'ruby on rails' or 'google cloud platform' is retained
mwe = []
for technology in technologies:
    text = nltk.regexp_tokenize(technology['name'], r'\s', gaps = True)
    if len(text) > 1:
        mwe.append(list(nltk.ngrams(text, len(text)))[0])

## Normalize

In [175]:
# Import job vacancy .json file to list of job vacancies called 'data'
json_filename = '20200422_developer_London_0.json'
with open (json_filename) as json_file:
    data = json.load(json_file)
    data = data['content']

# Normalize / tokenize
for jobad in data:
    # Make description lowercase
    jobad['description'] = jobad['description'].lower()

    # TODO: [ ] Test regex using nltk.concordance() to find context for each technology.  Are there any other split chars?
    # Tokenize by regexp to split words using [' ', '/', ',', '.'] chars as separators
    tokenized_text = nltk.regexp_tokenize(jobad['description'], r'[\s\/,.]', gaps=True)

    # Tokenize for multi-word expressions (mwe)
    mwe_tokenizer = nltk.tokenize.MWETokenizer(mwe, separator=" ")
    jobad['description'] = mwe_tokenizer.tokenize(tokenized_text)

    # Create list of technologies mentioned in jobad['description']
    jobad['description_tech'] = []
    subcategories = []
    for technology in technologies:
        for word in jobad['description']:
            if word == technology['name'] and word not in jobad['description_tech']:
                jobad['description_tech'].append(technology['subcategory'])

## Create co-occurrence matrix

In [176]:
# Create co-occurrence matrix
words = []
for technology in technologies:
    if technology['subcategory'] not in words:
        words.append(technology['subcategory'])

co_occurrence_matrix = {}
for word in words:
    co_occurrence_matrix[word] = {word : 0 for word in words}

for jobad in data:
    for word1 in jobad['description_tech']:
        for word2 in jobad['description_tech']:
            if word1 is not word2:
                co_occurrence_matrix[word1][word2] += 1

df = pd.DataFrame(co_occurrence_matrix)

## Create list of edges and nodes

In [177]:
# Edges

#add weights to edges
edge_list = [] #test networkx
for index, row in df.iterrows():
    i = 0
    for col in row:
        weight = float(col)/df.max().max()
        edge_list.append({ 'start' : index, 'finish' : df.columns[i], 'thickness' : weight })
        i += 1

#Remove edge if 0.0
edge_list = [x for x in edge_list if not x['thickness'] == 0.0]

#remove self references
for i in edge_list:
    if i['start'] == i['finish']:
        edge_list.remove(i)

In [178]:
node_list = []
for column in list(df.columns):
    node_list.append({
        'category' : next(technology['category'] for technology in technologies if technology['subcategory'] == column), 
        'subcategory' : column,
        'occurrences' : 0 ,
        'median_salary' : 0,
    })

for node in node_list:
    for jobad in data:
        if node['subcategory'] in jobad['description_tech']:
            node['occurrences'] += 1

    # Remove nodes with no edges (technologies that aren't mentioned in job ads)
    # if node['occurrences'] == 0:
    #    node_list.remove(node)

## Associate median salaries

In [179]:
getcontext().prec = 10
for node in node_list:
    salaries = []
    for jobad in data:
        if node['subcategory'] in jobad['description_tech'] and jobad['mid_salary'] is not None:
            salaries.append(jobad['mid_salary'])
    if salaries:
        # print(salaries)
        node.update(median_salary = int(statistics.median(salaries)))
    else:
        node.update(median_salary = "No vacancies advertised a salary.")

## Export nodes and edges to .csv format

In [181]:
with open (json_filename + 'flourish_nodes.csv', 'w') as outfile:
    fnames = ['technology', 'occurrences', 'category', 'subcategory', 'median_salary']
    writer = csv.DictWriter(outfile, fieldnames=fnames)
    writer.writeheader()
    for node in node_list:
        writer.writerow(node)

with open (json_filename + 'flourish_edges.csv', 'w') as outfile:
    fnames = ['start', 'finish', 'thickness']
    writer = csv.DictWriter(outfile, fieldnames=fnames)

    writer.writeheader()
    for edge in edge_list:
        writer.writerow(edge)